In [1]:
import os
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

import time
import glob
import tables as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce

2017-07-09 09:11:03


In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14

In [3]:
from invisible_cities.database import load_db

import invisible_cities.sierpe.blr as blr

import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.peak_functions as pf
import invisible_cities.reco.peak_functions_c as cpf

from   invisible_cities.reco.params           import S12Params 
from   invisible_cities.reco.params           import CalibVectors
from   invisible_cities.reco.params           import DeconvParams
from   invisible_cities.reco.params           import ThresholdParams

from   invisible_cities.core.system_of_units_c import units
from   invisible_cities.core.ic_types import minmax
import invisible_cities.icaro.mpl_functions as mpl
from   invisible_cities.icaro.pmaps_mpl import plot_s12
from   invisible_cities.icaro.pmaps_mpl import plot_s2si_map

In [4]:
import invisible_cities.icaro.mpl_functions as mpl

In [5]:
import invisible_cities.reco.event_model as evm
from invisible_cities.core.ic_types import minmax

In [6]:
from invisible_cities.evm.pmaps import S1
from invisible_cities.evm.pmaps import S2
from invisible_cities.evm.pmaps import S2Si

In [7]:
from invisible_cities.io.pmap_io                 import load_pmaps
from invisible_cities.io.pmap_io                 import s1_s2_si_from_pmaps

In [8]:
from invisible_cities.filters.s1s2_filter    import s1s2_filter
from invisible_cities.filters.s1s2_filter    import S12Selector

In [9]:
from invisible_cities.core.exceptions        import SipmEmptyList
from invisible_cities.core.exceptions        import SipmNotFound
from invisible_cities.core.exceptions        import PeakNotFound

In [10]:
from invisible_cities.core.configure     import configure

In [11]:
run_number = 0
DataPMT = load_db.DataPMT(run_number)
DataSiPM = load_db.DataSiPM(run_number)
xs = DataSiPM.X.values
ys = DataSiPM.Y.values
pmt_active = np.nonzero(DataPMT.Active.values)[0].tolist()

In [12]:
channel_id = DataPMT.ChannelID.values
coeff_blr = abs(DataPMT.coeff_blr.values)
coeff_c = abs(DataPMT.coeff_c .values)
adc_to_pes = abs(DataPMT.adc_to_pes.values)
adc_to_pes_sipm = abs(DataSiPM.adc_to_pes.values)

In [13]:
from invisible_cities.reco.dst_functions  import load_dst

In [14]:
from invisible_cities.reco.tbl_functions     import get_event_numbers_and_timestamps_from_file_name

In [15]:
filename = '/Users/jjgomezcadenas/Projects/IC/invisible_cities/database/test_data/KrMC_pmaps.h5'
h5 = tb.open_file(filename, "r") 
s1_dict, s2_dict, s2si_dict = load_pmaps(filename)

### s1_dict = {events:s1} and same for the other pmaps

In [16]:
print(set(s1_dict.keys()))

{23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 51}


In [17]:
print(set(s2_dict.keys()))

{32, 33, 34, 35, 36, 37, 38, 39, 40, 31}


In [18]:
print(set(s1_dict.keys()).intersection(set(s2_dict.keys())))

{32, 33, 34, 35, 36, 37, 38, 39, 40, 31}


In [19]:
event_numbers, timestamps = get_event_numbers_and_timestamps_from_file_name(filename)

In [20]:
print(event_numbers)

[23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 49 51]


In [21]:
assert (set(event_numbers) == set(s1_dict.keys()).union(set(s2_dict.keys())))

### Follow the Dorothea loop for the first event

In [22]:
evt_number = event_numbers[0]

In [23]:
s1, s2, s2si = s1_s2_si_from_pmaps(s1_dict, s2_dict, s2si_dict, evt_number)

In [24]:
s1

S1 (number of peaks = 1)
peak number = 0: Peak(samples = 9 width =      0.2 mus , energy =      8.4 pes
        height =      2.3 pes tmin-tmax = minmax(min=100.075, max=100.3) mus  

In [25]:
s2 == None

True

In [26]:
s2si

In [37]:
for i, evt_number in enumerate(event_numbers):
    s1, s2, s2si = s1_s2_si_from_pmaps(s1_dict, s2_dict, s2si_dict, evt_number)
    if s1 and s2 and s2si:
        break
print (i,evt_number)

8 31


#### Event 23 is not present in s2_dict, thus the result is an empty dict (same for s2si)

### Configure Dorothea

In [27]:
conf = configure('dummy /Users/jjgomezcadenas/Projects/IC/invisible_cities/config/dorothea.conf'.split()).as_dict

In [28]:
conf

{'compression': 'ZLIB4',
 'config_file': '/Users/jjgomezcadenas/Projects/IC/invisible_cities/config/dorothea.conf',
 'drift_v': 0.001,
 'file_out': '/tmp/KrDST.h5',
 'files_in': '$ICDIR/database/test_data/KrMC_pmaps*.h5',
 'full_files': False,
 'hide_config': False,
 'nmax': 1,
 'no_files': False,
 'no_overrides': False,
 'nprint': 1,
 'print_config_only': False,
 'run_all': False,
 'run_number': 0,
 's1_emax': 20,
 's1_emin': 0,
 's1_ethr': 0.5,
 's1_hmax': 20,
 's1_hmin': 0,
 's1_lmax': 20,
 's1_lmin': 4,
 's1_nmax': 1,
 's1_nmin': 1,
 's2_emax': 20000,
 's2_emin': 1000,
 's2_ethr': 0.5,
 's2_hmax': 10000,
 's2_hmin': 0,
 's2_lmax': 1000,
 's2_lmin': 1,
 's2_nmax': 1,
 's2_nmin': 1,
 's2_nsipmmax': 100,
 's2_nsipmmin': 1,
 'skip': 0,
 'verbosity': 0}

In [29]:
mm = units.mm
mus = units.mus
pes = units.pes

In [30]:
conf.update(dict(run_number = 0,

                     drift_v     =      1 * mm / mus,
                     s1_nmin     =      1,
                     s1_nmax     =      1,
                     s1_emin     =      0 * pes,
                     s1_emax     =     30,
                     s1_lmin     =      4,
                     s1_lmax     =     20,
                     s1_hmin     =    0.5 * pes,
                     s1_hmax     =     10 * pes,
                     s1_ethr     =    0.5 * pes,
                     s2_nmin     =      1,
                     s2_nmax     =      2,
                     s2_emin     =    1e3 * pes,
                     s2_emax     =    1e8 * pes,
                     s2_lmin     =      1,
                     s2_lmax     =     20,
                     s2_hmin     =    500 * pes,
                     s2_hmax     =    1e5 * pes,
                     s2_ethr     =      1 * pes,
                     s2_nsipmmin =      2,
                     s2_nsipmmax =   1000,
                     nmax        = 10))

In [31]:
conf

{'compression': 'ZLIB4',
 'config_file': '/Users/jjgomezcadenas/Projects/IC/invisible_cities/config/dorothea.conf',
 'drift_v': 0.001,
 'file_out': '/tmp/KrDST.h5',
 'files_in': '$ICDIR/database/test_data/KrMC_pmaps*.h5',
 'full_files': False,
 'hide_config': False,
 'nmax': 10,
 'no_files': False,
 'no_overrides': False,
 'nprint': 1,
 'print_config_only': False,
 'run_all': False,
 'run_number': 0,
 's1_emax': 30,
 's1_emin': 0.0,
 's1_ethr': 0.5,
 's1_hmax': 10.0,
 's1_hmin': 0.5,
 's1_lmax': 20,
 's1_lmin': 4,
 's1_nmax': 1,
 's1_nmin': 1,
 's2_emax': 100000000.0,
 's2_emin': 1000.0,
 's2_ethr': 1.0,
 's2_hmax': 100000.0,
 's2_hmin': 500.0,
 's2_lmax': 20,
 's2_lmin': 1,
 's2_nmax': 2,
 's2_nmin': 1,
 's2_nsipmmax': 1000,
 's2_nsipmmin': 2,
 'skip': 0,
 'verbosity': 0}

### Init the s12 selector

In [32]:
s1s2_selector = S12Selector(**conf)

### Filter step

In [38]:
f = s1s2_filter(s1s2_selector, s1, s2, s2si)

In [39]:
f

True